In [ ]:
install.packages(c("listenv", "future"))

In [ ]:

library(parallel)
library(doParallel)
library(future)
library(iterators)
library(listenv)

options(future.globals.maxSize = 8 * 1024^3) # increase dataset size limit taken by future to 8GB

# detects number of cores
n.cores <- parallel::detectCores()


plan(list(
  tweak(multisession, workers = ((n.cores + 5) %/% 3) %/% 2),
  tweak(multisession, workers = ((n.cores + 5) %/% 3) %/% 2),
  tweak(multisession, workers = 3)
))


In [ ]:
start.time <- Sys.time()

In [4]:
input_dir <- getwd()

In [5]:
source(paste(getwd(),"/Workflow_R_Functions.r", sep = ""))

In [9]:
input_table <- read.csv(paste(input_dir,"/input_table.csv", sep = ""))
input_table

X.1,X,mzml_files,ResultFileNames,File_id
<int>,<int>,<chr>,<chr>,<chr>
1,1,./VN_211016_acetyl_carnitine.mzML,./VN_211016_acetyl_carnitine,file_1
2,2,./VN_211016_betaine.mzML,./VN_211016_betaine,file_2
3,3,./VN_211016_butanoyl_carnitine.mzML,./VN_211016_butanoyl_carnitine,file_3
4,4,./VN_211016_cyst_acid_PRM_neg.mzML,./VN_211016_cyst_acid_PRM_neg,file_4
5,5,./VN_211016_cyst_acid_PRM_pos.mzML,./VN_211016_cyst_acid_PRM_pos,file_5
6,6,./VN_211016_isovalerylcarnitine.mzML,./VN_211016_isovalerylcarnitine,file_6
7,7,./VN_211016_methionine_sulfoxide.mzML,./VN_211016_methionine_sulfoxide,file_7
8,8,./VN_211016_pipecolate.mzML,./VN_211016_pipecolate,file_8
9,9,./VN_211016_propanoyl_carnitine.mzML,./VN_211016_propanoyl_carnitine,file_9


In [13]:
input_table_idxs <- listenv()

Rows: 12 Columns: 12
── Column specification ──────────────────────────────────────────────────────
Delimiter: "\t"
chr (3): molecularFormula, adduct, precursorFormula
dbl (9): rank, SiriusScore, TreeScore, IsotopeScore, numExplainedPeaks, expl...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [23]:
#run_metfrag(paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/insilico/metparam_list.txt", sep = ""), "."), sep =""))

In [ ]:
for (i in 1:nrow(input_table)){
    input_table_idxs[[i]] <- future({
        
        sirius_adduct(input_dir, 
                             x = input_table[i, "ResultFileNames"], 
                             SL = FALSE)
        

        metfrag_param(x = paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/insilico/MS1DATAsirius.csv", sep = ""), "."), sep =""),
                      result_dir = input_table[i, "ResultFileNames"], 
                      input_dir, 
                      sl_mtfrag = NA, 
                      SL = FALSE, 
                      ppm_max = 5, 
                      ppm_max_ms2= 15)
        
        run_metfrag(paste(input_dir, str_remove(paste(input_table[i, "ResultFileNames"], "/insilico/metparam_list.txt", sep = ""), "."), sep =""))
        
    })
    
}

In [ ]:
input_table_idxs <- as.list(input_table_idxs)
v_input_table_idxs <- future::value(input_table_idxs)

In [ ]:
end.time <- Sys.time()

In [ ]:
time.taken <- end.time - start.time
print(time.taken)